![](https://i.imgur.com/eBRPvWB.png)

# Klasyfikacja nazwisk z użyciem sieci rekurencyjnych przetwarzających litery

Będziemy budować i trenować klasyfikator RNN przetwarzający znaki, aby klasyfikował słowa. Znakowy RNN odczytuje słowa jako serię znaków - wyprowadzając prognozę i "stan ukryty" na każdym kroku, wprowadzając swój poprzedni stan ukryty do każdego kolejnego kroku. Końcową prognozę potraktujemy jako wynik, tj. klasę, do należy dane słowo.

Mówiąc dokładniej, przeprowadzimy trening sieci na kilku tysiącach nazwisk pochodzących z 18 języków, i będziemy rozpoznawać język z którego nazwisko pochodzi:

```
> Czarnowski
(79.13%) Polish
( 6.21%) Russian
( 6.04%) Czech

> Satoshi
(47.51%) Japanese
(26.99%) Arabic
(10.29%) Polish
```

# Polecana lektura

Zakładam że jest już zainstalowany PyTorch, znasz Python'a, oraz znasz pojęcie Tensor'ów:

* http://pytorch.org/ - instalacja PyTorch
* [Deep Learning with PyTorch: A 60-minute Blitz](http://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html) - Podstawy PyTorch
* [jcjohnson's PyTorch examples](https://github.com/jcjohnson/pytorch-examples) przykłady wykorzystania PyTorch
* [Introduction to PyTorch for former Torchies](https://github.com/pytorch/tutorials/blob/master/Introduction%20to%20PyTorch%20for%20former%20Torchies.ipynb) jeżeli znasz Lua Torch

Trochę wiedzy o RNN:

* [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) przykłady z życia wzięte
* [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) RNN i LSTM w pigułce

# Przygotowanie danych

Dane znajdują się w folderze `data/names` są podzielone na 18 plików tekstowych nazwanych wg schematu "[Language].txt". Każdy plik zawiera wiele nazwisk, po 1 w każdej linii w losowej kolejności, w większości zapisanych alfabetem łacińskim.

Otrzymamy coś w rodzaju słownika (python dictionary) składającego się z list nazwisk dla każdego z języków, `{language: [names ...]}`. Ogólne zmienne "category" oraz "line" (dla języka i nazwiska w naszym przypadku) są używane dla zapewnienia przyszłej rozszerzalności.

In [0]:
import glob

all_filenames = glob.glob('data/names/*.txt')
print(all_filenames)

In [0]:
# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename).read().strip().split('\n')
    return lines

for filename in all_filenames:
    category = filename.split('/')[-1].split('.')[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)
print('n_categories =', n_categories)

In [0]:
all_letters = ''.join(sorted(list(set(''.join([''.join(v) for k,v in category_lines.items()])))))
n_letters = len(all_letters)

print(f'all_letters = {all_letters}')
print(f'n_letters = {n_letters}')

Teraz mamy "category_lines", słownik mapujący każdą kategorię (język) do listy nazwisk. Przechowujemy także "all_categories" (po prostu lsitę języków) oraz "n_categories" dla przyszłego użytku.

In [0]:
c = category_lines['Polish']; print(len(c))
print(c[:5])

# Konwersja nazwisk na tensory

Teraz jak już wszystkie nazwiska są poukładane, musimy je przekonwertować na tensory.

Do reprezentacji pojedynczej litery używamy kodowania 1-z-n (ang. one-hot vector) o rozmiarze <1 x n_letters>. Wektor 1-z-n jest wypełniony zerami za wyjątkiem 1 znajdującej w miejscu przynależnym do pożądanej litery np. `"b" = <0 1 0 0 0 ...>`.

Aby zbudować słowo łączymy kilka takich wektorów w pojedynczą macierz o wymiarach: `<line_length x 1 x n_letters>`.

Dodatkowy wymiar oznaczony 1 jest potrzebny ponieważ PyTorch zakłada że wszystko jest ładowane porcjami (ang. batch) - my tu używamy porcji o rozmiarze równym 1.


In [0]:
import torch

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letter_to_tensor(letter):
    tensor = torch.zeros(1, n_letters)
    letter_index = all_letters.find(letter)
    tensor[0][letter_index] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def line_to_tensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        letter_index = all_letters.find(letter)
        tensor[li][0][letter_index] = 1
    return tensor

In [0]:
print(letter_to_tensor('c'))

In [0]:
print(line_to_tensor('Jones').size())

# Budowa sieci rekurencyjnej

Przed zastosowaniem metody autograd, stworzenie RNN w Torchu wiązało się z klonowaniem parametrów warstwy dla kolejnych kroków. Warstwy przechowywały stany ukryte oraz gradienty, które obecnie są całkowicie obsługiwane przez graf. Oznaczna to że możesz zaimplementować RNN w bardzo przystępnej formie podobnie jak zwyczajne wartwy feed-forward.

Ten moduł RNN (zainspirowany [the PyTorch for Torch users tutorial](https://pytorch.org/tutorials/beginner/former_torchies/nn_tutorial.html#example-2-recurrent-net)) to tylko 2 linearne warstwy, które operują na wejściu oraz stanie ukrytym, a także aplikują warstwę LogSoftmax na wyjściu.

![](https://i.imgur.com/Z2xbySO.png)

In [0]:
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=-1)
    
    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(1, self.hidden_size))

## Ręczne testowanie sieci

Mając zdefiniowaną naszą klasę `RNN`, stwórzmy jej nową instancję:

In [0]:
n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)
print(rnn)

Aby uruchomić jakiś krok naszej sieci, musimy najpierw przekazać jakieś dane wejściowe (w tym przypadku Tensor bieżącej litery) oraz poprzedni stan ukryty (który inicjalizujemy na początku zerami). W taki sposób otrzymamy wynik (prawdopodobieństwo każdego języka) oraz następny stan ukryty (niezbędny w kolejnym kroku).

Pamiętaj że moduły PyTorch operują na Zmiennych (Variables), a nie bezpośrednio na Tensorach.


In [0]:
input = Variable(letter_to_tensor('A'))
hidden = rnn.init_hidden()

output, next_hidden = rnn(input, hidden)
print('output.size =', output.size())

Na potrzeby wydajności nie chcemy tworzyć nowego Tensora dla każdego kroku, więc używamy metody `line_to_tensor` zamiast `letter_to_tensor` a także tzw. wycinków (slices). Można to jeszcze zoptymalizować obliczając wcześniej porcje (batches) Tensorów.

In [0]:
input = Variable(line_to_tensor('Albert'))
hidden = Variable(torch.zeros(1, n_hidden))

output, next_hidden = rnn(input[0], hidden)
output

Tak jak widać wynikiem jest Tensor `<1 x n_categories>`, gdzie każdy element jest prawdopodobieństwem danej kategorii.

# Przygotowanie do treningu sieci

Zanim rozpoczeniemy trening musimy zaimplementować kilka funkcji pomocniczych.
Pierwsza to interpreter wyjść naszej sieci. Użyjemy tu `Tensor.topk` aby uzyskać indeks największej wartości:

In [0]:
def category_from_output(output):
    top_n, top_i = output.data.topk(1) # Tensor out of Variable with .data
    category_i = top_i[0][0]
    return all_categories[category_i], category_i

print(category_from_output(output))

Potrzebna będzie także metoda otrzymywania przykładów treningowych (nazwisko oraz język):


In [0]:
import random

def random_training_pair():                                                                                                               
    category = random.choice(all_categories)
    line = random.choice(category_lines[category])
    category_tensor = Variable(torch.LongTensor([all_categories.index(category)]))
    line_tensor = Variable(line_to_tensor(line))
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = random_training_pair()
    print('category =', category, '/ line =', line)

# Trening sieci

Aby wytrenować sieć musimy jej "pokazać" wiele przykładów, pozwolić jej zgadywać i powiedzieć jej, kiedy się myli.

Na funkcję straty nadaje się metoda: [`nn.NLLLoss`](http://pytorch.org/docs/nn.html#nllloss), ponieważ ostatnia wartwa RNN to `nn.LogSoftmax`.

In [0]:
criterion = nn.NLLLoss()

Dodatkowo tworzymy 'optimizer', który będzie aktualizował parametry naszego modelu zależnie od gradientów. Wykorzystujemy tu zwykły algorytm SGD z małym tempem uczenia.

In [0]:
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn
optimizer = torch.optim.SGD(rnn.parameters(), lr=learning_rate)

W każdej pętli treningu:

* Tworzymy wejściowe i oczekiwane tensory
* Tworzymy zerowy stan wejściowy
* Odczytujemy każdą literę
    * przechowujemy stan ukryty dla następnej litery
* Porównujemy otrzymany wynik z wartością oczekiwaną
* Robimy wsteczną propagację
* Zwracamy wynik i stratę

In [0]:
def train(category_tensor, line_tensor):
    rnn.zero_grad()
    hidden = rnn.init_hidden()
    
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    optimizer.step()

    return output, loss.item()

Teraz wystarczy to powtórzyć dla wielu próbek. Skoro funkcja `train` zwraca wynik i stratę można wypisać jej predykcje i śledzić wartości straty. Ponieważ mamy tysiące próbek wyświetlamy wyniki tylko co `print_every` kroków i liczymy średnią stratę.


In [0]:
import time
import math

n_epochs = 100000
print_every = 5000
plot_every = 1000

# Keep track of losses for plotting
current_loss = 0
all_losses = []

def time_since(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for epoch in range(1, n_epochs + 1):
    # Get a random training input and target
    category, line, category_tensor, line_tensor = random_training_pair()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss
    
    # Print epoch number, loss, name and guess
    if epoch % print_every == 0:
        guess, guess_i = category_from_output(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (epoch, epoch / n_epochs * 100, time_since(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if epoch % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

# Kreślenie wyników

Kreślenie historii straty z funkcji `all_losses` pokazuje przebieg uczenia sieci:

In [0]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

import matplotlib.style
import matplotlib as mpl
mpl.style.use('seaborn')

fig=plt.figure()
aaa = plt.plot(all_losses)

In [0]:
y = all_losses
n = 10
averages = []
for i in range(len(y) - n):
    window = y[i:i+n]
    avg = sum(window) / n
    # print(window, avg)
    averages.append(avg)

In [0]:
len(all_losses), len(averages)

In [0]:
fig=plt.figure()
plt.plot(averages);

# Ocenianie wyników

Aby sprawdzić jak dobrze sieć radzi sobie z różnymi kategoriami tworzymy tablicę pomyłek, w której dla każdego języka (wiersz) pokazane są wyniki predykcji (kolumny). Aby wyliczyć tablicę pomyłek przepuszczamy serię próbek przez sieć funkcją  `evaluate()`,  która robie to samo co  `train()` za wyjątkiem wstecznej propagacji.

In [0]:
# Keep track of correct guesses in a confusion matrix
confusion = torch.zeros(n_categories, n_categories)
n_confusion = 10000

# Just return an output given a line
def evaluate(line_tensor):
    hidden = rnn.init_hidden()
    
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
    
    return output

# Go through a bunch of examples and record which are correctly guessed
for i in range(n_confusion):
    category, line, category_tensor, line_tensor = random_training_pair()
    output = evaluate(line_tensor)
    guess, guess_i = category_from_output(output)
    category_i = all_categories.index(category)
    confusion[category_i][guess_i] += 1
    
# Normalize by dividing every row by its sum
for i in range(n_categories):
    confusion[i] = confusion[i] / confusion[i].sum()

In [0]:
mpl.style.use('default')
mpl.style.use('seaborn-ticks')

mpl.pyplot.set_cmap('hot_r')

# Set up plot
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion.numpy())
fig.colorbar(cax)

# Set up axes
ax.set_xticklabels([''] + all_categories, rotation=90)
ax.set_yticklabels([''] + all_categories)

# Force label at every tick
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

plt.show()

Wybierz widoczne kwadraciki poza główną przekątną aby zobaczyć, które przewidywania nie są prawidłowe, np. chiński zamiast koreańskiego czy hiszpański zamiast włoskiego. Widać, że klasyfikuje dobrze grecki, a słabo angielski (prawdopodobnie z powodu nakładania się z innymi językami).


# Klasyfikacja podanych nazwisk

In [0]:
import numpy as np

def predict(input_line, n_predictions=3):
    print('\n> %s' % input_line)
    output = evaluate(Variable(line_to_tensor(input_line)))

    # Get top N categories
    topv, topi = output.data.topk(n_predictions, 1, True)
    predictions = []

    for i in range(n_predictions):
        value = topv[0][i]
        category_index = topi[0][i]
        pred = np.exp(value)
        print('(%.2f%%) %s' % (pred*100, all_categories[category_index]))
        # print('(%.2f) %s' % (value, all_categories[category_index]))
        predictions.append([value, all_categories[category_index]])

predict('Dovesky')
predict('Jackson')
predict('Satoshi')
predict('Czarnowski')
predict('Kazimierczak')
predict('Wołk')

# Ćwiczenia

* Wypróbuj inny zbiór danych typu linia -> kategoria, np.:
    * słowo -> język
    * imię -> płeć
    * bohater -> pisarz
   
* Popraw wyniki tworząc większą sieć i/lub sieć o lepszej strukturze
    * Więcej linearnych warstw
    * Wypróbuj warstwy `nn.LSTM` oraz `nn.GRU` 
    * Połącz wiele takich sieci RNN w sieć wyższego rzędu